In [15]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf

In [16]:
batch_size = 32
img_height = 16
img_width = 8

train_ds = tf.keras.utils.image_dataset_from_directory(
    "Dataset",
    validation_split=0.3,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "Dataset",
    validation_split=0.3,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)
    
normalization_layer = tf.keras.layers.Rescaling(1./255)

Found 6598 files belonging to 42 classes.
Using 4619 files for training.
Found 6598 files belonging to 42 classes.
Using 1979 files for validation.


In [17]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

0.08627451 0.6313726


In [18]:
num_classes = len(os.listdir("Dataset"))

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(196, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  # tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(num_classes, activation=tf.nn.softmax)
])

In [19]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=5*1e-6)
# model.compile(optimizer=opt, 
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

from tensorflow.keras.callbacks import TensorBoard
import random
NAME = "basic"+str(random.randint(1,1000))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
# cd "C:\src\Python\Smart Clicker\V9\"
# tensorboard --logdir="logs/"

In [53]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50, callbacks=[tensorboard]
)


Epoch 1/50
145/145 [==============================] - 1s 3ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.0013 - val_accuracy: 0.9995
Epoch 2/50
145/145 [==============================] - 0s 3ms/step - loss: 0.0516 - accuracy: 0.9866 - val_loss: 0.0300 - val_accuracy: 0.9934
Epoch 3/50
145/145 [==============================] - 0s 3ms/step - loss: 0.0065 - accuracy: 0.9991 - val_loss: 0.0022 - val_accuracy: 0.9995
Epoch 4/50
145/145 [==============================] - 0s 3ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.0021 - val_accuracy: 0.9995
Epoch 5/50
145/145 [==============================] - 0s 3ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0021 - val_accuracy: 0.9995
Epoch 6/50
145/145 [==============================] - 1s 4ms/step - loss: 9.9273e-04 - accuracy: 1.0000 - val_loss: 0.0019 - val_accuracy: 0.9995
Epoch 7/50
145/145 [==============================] - 0s 3ms/step - loss: 8.9042e-04 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 0.999

In [26]:
val_loss, val_acc = model.evaluate(val_ds)



62/62 [==============================] - 0s 1ms/step - loss: 0.0014 - accuracy: 0.9995


In [66]:
import numpy as np
import cv2
X=[]
y=[]

for index, batch in enumerate(val_ds):
    X, y = batch
    X = X.numpy()
    y = y.numpy()
    
    y_predict = model.predict(X)
    y_predict = np.argmax(y_predict, axis=1)
    for i in range(32):
        if y[i]!=y_predict[i]:
            print(index, i)
            cv2.imwrite(f"{random.randint(1, 100)}.png", X[i])
    # print(y, y.shape, y_predict.shape, len(val_ds))
    # print(y_predict)
    # break


35 2


IndexError: index 27 is out of bounds for axis 0 with size 27